In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [55]:
df = pd.read_parquet('./../data/fhv_tripdata_2021-01.parquet')

df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
#distribution of duration
df_one_hour = df[(df.duration >= 1) & (df.duration <=60)]

#    df_one_hour.PUlocationID.fillna(-1, inplace=True), df_one_hour.DOlocationID.fillna(-1, inplace=True)

In [56]:
df.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


In [62]:
df.shape

(1154112, 8)

In [57]:
#average duration
df.duration.mean()

19.167224093791006

In [58]:
df_one_hour

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [ ]:
fig, ax = plt.subplots()
#sns.histplot(x='duration', data=df, ax=ax)

In [59]:
#Percentage of missing values
df_one_hour.PUlocationID.isna().sum()/len(df_one_hour)*100,\
df_one_hour.DOlocationID.isna().sum()/len(df_one_hour)*100

(83.52732770722618, 13.327044059158824)

In [20]:
# df_one_hour_wo_na = df_one_hour[(~df_one_hour.PUlocationID.isna()) & (~df_one_hour.DOlocationID.isna())]
# df_one_hour_wo_na

In [60]:
df_one_hour.PUlocationID.fillna(-1, inplace=True), df_one_hour.DOlocationID.fillna(-1, inplace=True)

/tmp/ipykernel_7524/1731950699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hour.PUlocationID.fillna(-1, inplace=True), df_one_hour.DOlocationID.fillna(-1, inplace=True)


(None, None)

In [61]:
df_one_hour

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


# Model 

In [48]:
def pre_processing(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    #distribution of duration
    df_one_hour = df[(df.duration >= 1) & (df.duration <=60)]

    df_one_hour.PUlocationID.fillna(-1, inplace=True), df_one_hour.DOlocationID.fillna(-1, inplace=True)

    #one hot encoding

    categorical = ['PUlocationID', 'DOlocationID']

    df_one_hour[categorical] = df_one_hour[categorical].astype(str)
    
    return df_one_hour

In [49]:
df_one_hour = pre_processing('./../data/fhv_tripdata_2021-01.parquet')

/tmp/ipykernel_7524/352650444.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hour.PUlocationID.fillna(-1, inplace=True), df_one_hour.DOlocationID.fillna(-1, inplace=True)
/tmp/ipykernel_7524/352650444.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hour[categorical] = df_one_hour[categorical].astype(str)


In [50]:
#Model training

lr = LinearRegression()
train_dicts = df_one_hour[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_one_hour[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107202403

In [64]:
X_train.shape

(1109826, 525)

In [51]:
#validation set
df_val = pre_processing('./../data/fhv_tripdata_2021-02.parquet')

/tmp/ipykernel_7524/352650444.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hour.PUlocationID.fillna(-1, inplace=True), df_one_hour.DOlocationID.fillna(-1, inplace=True)
/tmp/ipykernel_7524/352650444.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hour[categorical] = df_one_hour[categorical].astype(str)


In [52]:
#Error on validation set

test_dicts = df_val[categorical].to_dict(orient='records')

X_test = dv.transform(test_dicts)

target = 'duration'
y_test = df_val[target].values

y_pred = lr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

11.014283118149931